## Customer Dataset Cleaning

In [ ]:
%pip install fuzzywuzzy
!python -m pip install --upgrade pip
%pip install python-Levenshtein
%pip install pandas numpy matplotlib seaborn scikit-learn jupyter

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached matplotlib-3.10.7-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
Using cached matplotlib-3.10.7-cp312-cp312-win_amd64.whl (8.1 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl (8.7 MB)
Using cached jupyter-1.1.1-py2.py3-none-any.whl (2.7 kB)
Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl (226 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Usin

In [ ]:
# Activate kernal:
# .\venv_fyp\Scripts\activate

# Step 1: Import libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from fuzzywuzzy import process, fuzz
import pycountry


In [3]:
# Step 2: Load dataset
# Replace 'customer_dataset.csv' with your actual file name or path
file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun/2021 - 2025 Customer.csv"

original_dataset_name = "2021 - 2025 Customer.csv"

# Read dataset
df = pd.read_csv(file_path)

# Show first few rows
df.head()

,CustomerID,Date of Birth,Gender,Income Level,City,State,Country
0,CUST0001,NaN,female,NaN,Sungai Besi,Kuala Lumpur,Malaysia
1,CUST0001,NaN,female,NaN,NaN,NaN,NaN
2,CUST0002,NaN,female,NaN,Mutiara Damansara,Selangor,Malaysia
3,CUST0002,NaN,female,NaN,NaN,NaN,NaN
4,CUST0003,NaN,female,NaN,Shah Alam,Selangor,Malaysia


### Data Profiling

In [ ]:
## Create later

### Initial checking: (Before perform data cleaning) Check optional and mandatory columns 

In [ ]:
# Optional columns
def check_optional_columns(df, threshold=0.5):
    """
    Check optional columns for fill percentage and drop columns that are mostly empty.
    Returns the modified DataFrame and a friendly message.
    """
    
    optional_columns = ["Date of Birth", "Gender", "Income Level"]

    # Normalize column names
    df.columns = df.columns.str.strip().str.lower()
    optional_columns = [col.lower() for col in optional_columns]

    dropped_columns = []

    for col in optional_columns:
        if col in df.columns:
            fill_ratio = df[col].count() / len(df) # 
            if fill_ratio < threshold:
                dropped_columns.append(col)
                df.drop(columns=[col], inplace=True)  # Drop the column immediately
                # df[col].count(): This counts the number of non-missing (non-null/non-NaN) values in the current column (col).
                # len(df): This gives the total number of rows in the DataFrame.
                # fill_ratio: The division calculates the proportion of filled (non-missing) values in that column. A ratio of 1.0 means the column is entirely filled; a ratio of 0.1 means 90% of the values are missing.


    # Generate user-friendly message
    if dropped_columns:
        dropped_str = ", ".join(dropped_columns)
        message = (
            f"We noticed that very few entries were provided for {dropped_str}. "
            "These columns have been removed. "
            "Segmentation will still be performed using geographic (City, State, Country) "
            "and behavioral data (e.g., orders, purchase items, total spend)."
        )
    else:
        message = "All optional columns have enough data and are kept for analysis."
    
    return df, message

# Mandatory columns 
def check_mandatory_columns(df, threshold=0.2):
    """
    Check mandatory columns for missing values (column-wise).
    Does not drop columns — only warns user if any column is too incomplete.
    Returns the DataFrame and a message summarizing issues.
    """

    mandatory_columns = ["CustomerID", "City", "State", "Country"]

    # Normalize column names
    df.columns = df.columns.str.strip().str.lower()
    mandatory_columns = [col.lower() for col in mandatory_columns]

    missing_report = []
    warning_columns = []

    for col in mandatory_columns:
        if col in df.columns:
            fill_ratio = df[col].count() / len(df)
            missing_percent = (1 - fill_ratio) * 100

            missing_report.append(f"{col}: {missing_percent:.1f}% missing")

            # Warn if missing exceeds threshold
            if fill_ratio < (1 - threshold):
                warning_columns.append(col)
        else:
            # Handle case where column completely missing
            missing_report.append(f"{col}: column not found (100% missing)")
            warning_columns.append(col)

    # Generate friendly message
    if warning_columns:
        warning_str = ", ".join(warning_columns)
        message = (
            f"Some key fields have a high number of missing values: {warning_str}. "
            "The system will still continue cleaning and processing, "
            "but missing values will be handled automatically by our system. "
            "Please ensure your source data is as complete as possible for more accurate segmentation results.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )
    else:
        message = (
            "All mandatory columns have sufficient data and are ready for cleaning.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )

    return df, message


In [5]:
# --- Apply checks and save cleaned dataset ---
# Step 1: Optional columns
df, optional_check_message = check_optional_columns(df)

# Step 2: Mandatory columns (just check, but keep all columns)
df, mandatory_check_message = check_mandatory_columns(df)

# Step 3: Save the updated dataset for cleaning

# Split the name and extension
base_name, ext = os.path.splitext(original_dataset_name)

# Create new cleaned file name
cleaned_file = f"{base_name}_cleaned{ext}"

# Save the cleaned dataset
df.to_csv(cleaned_file, index=False)

# --- Display results ---
print("Optional Columns Check:")
print(optional_check_message)

print("\nMandatory Columns Check:")
print(mandatory_check_message)

print(f"\nCleaned dataset saved as '{cleaned_file}'")


Optional Columns Check:
We noticed that very few entries were provided for date of birth, income level. These columns have been removed. Segmentation will still be performed using geographic (City, State, Country) and behavioral data (e.g., orders, purchase items, total spend).

Mandatory Columns Check:
Some key fields have a high number of missing values: city, state, country. The system will still continue cleaning and processing, but missing values will be handled automatically by our system. Please ensure your source data is as complete as possible for more accurate segmentation results.

Missing Data Summary:
customerid: 0.0% missing
city: 44.8% missing
state: 46.9% missing
country: 44.7% missing

Cleaned dataset saved as '2021 - 2025 Customer_cleaned.csv'


In [6]:
# After perform initial checking on optional and mandatory columns
df.head()

,customerid,gender,city,state,country
0,CUST0001,female,Sungai Besi,Kuala Lumpur,Malaysia
1,CUST0001,female,NaN,NaN,NaN
2,CUST0002,female,Mutiara Damansara,Selangor,Malaysia
3,CUST0002,female,NaN,NaN,NaN
4,CUST0003,female,Shah Alam,Selangor,Malaysia


### Perform Data Cleaning - CustomerDataset

In [ ]:
# ============================================= STAGE 3: STANDARDIZATION & NORMALIZATION =============================================

def normalize_columns_name(df):
    """Normalize column names: lowercase, strip spaces"""
    df.columns = df.columns.str.strip().str.lower()
    return df

# ===============================================================================

def standardize_customer_id(df):
    """Standardize CustomerID format"""
    if 'customerid' in df.columns:
        df['customerid'] = df['customerid'].astype(str).str.strip().str.upper()
    return df

# ===============================================================================

def standardize_dob(df):
    """Standardize Date of Birth column and convert to YYYY-MM-DD"""
    if 'dob' in df.columns:
        def parse_date(x):
            if pd.isnull(x):
                return np.nan
            for format in ("%d/%m/%Y", "%m-%d-%y", "%Y-%m-%d", "%d-%b-%Y", "%d-%m-%Y"):    
                try:
                    return datetime.strptime(str(x), format).date() # Final format: YYYY-MM-DD | 2025-10-15
                except Exception:
                    continue
            return np.nan  # If no valid format found
        df['dob'] = df['dob'].apply(parse_date)
    return df

# %d/%m/%Y → 12/05/2000
# %m-%d-%y → 05-12-00
# %Y-%m-%d → 2000-05-12
# %d-%b-%Y → 12-May-2000
# %d-%m-%Y → 12-5-2000

# ===============================================================================

def derive_age_features(df):
    """Derive Age from DOB"""
    if 'dob' in df.columns:
        today = date.today()
        df['age'] = df['dob'].apply(
            lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day))
            if pd.notnull(x) else np.nan
        )
    return df
# Example: ((today.month, today.day) < (x.month, x.day))
# (10,15) < (12,1) → True (birthday in Dec is after Oct 15)
# (10,15) < (10,16) → True (birthday tomorrow)
# (10,15) < (5,20) → False (birthday already passed)

# This function calculates each person’s age from their date of birth (dob) by subtracting their birth year from the current year and adjusting if their birthday hasn’t occurred yet this year.

# ===============================================================================

def derive_age_group(df):
    """Derive Age Group based on defined buckets"""
    if 'age' in df.columns:
        def categorize_age(age):
            if pd.isnull(age):
                return 'Unknown'
            if age < 18: return 'Below 18'
            elif 18 <= age <= 24: return '18-24'
            elif 25 <= age <= 34: return '25-34'
            elif 35 <= age <= 44: return '35-44'
            elif 45 <= age <= 54: return '45-54'
            elif 55 <= age <= 64: return '55-64'
            else: return 'Above 65'
        df['age_group'] = df['age'].apply(categorize_age)
    return df

# =================================================================================

def standardize_gender(df):
    """Clean and standardize gender values"""
    if 'gender' in df.columns:
        # Clean text (remove spaces, make lowercase)
        df['gender'] = df['gender'].astype(str).str.strip().str.lower()

        # Standardize using keyword detection
        def detect_gender(value):
            if any(word in value for word in ['m', 'male', 'man', 'boy']):
                return 'Male'
            elif any(word in value for word in ['f', 'female', 'woman', 'girl']):
                return 'Female'
            else:
                return 'Unknown'

        df['gender'] = df['gender'].apply(detect_gender)
    return df

# ==================================================================================

def standardize_location(df):
    """Standardize City, State, Country using fuzzy and pycountry"""
    #  City 
    if 'city' in df.columns:
        df['city'] = df['city'].astype(str).str.title().str.strip()

    # State 
    if 'state' in df.columns:
        states = [sub.name for sub in pycountry.subdivisions if sub.country_code == 'MY']
        df['state'] = df['state'].astype(str).str.title().str.strip()
        df['state'] = df['state'].apply(
            lambda x: process.extractOne(x, states, scorer=fuzz.token_sort_ratio)[0] if x else 'Unknown'
        )

    # Country ---
    if 'country' in df.columns:
        countries = [c.name for c in pycountry.countries]
        df['country'] = df['country'].astype(str).str.title().str.strip()
        df['country'] = df['country'].apply(
            lambda x: process.extractOne(x, countries, scorer=fuzz.token_sort_ratio)[0]
            if x and len(x) > 2 else 'Malaysia'
        )
    return df

# =============================================(WIP) STAGE 4: MISSING VALUE HANDLING =============================================

def handle_missing_values_row_aware(df, min_cols_filled=2, city_to_state=None):
    """
    Handle missing values in a row-aware way.

    Parameters:
    - df: pandas DataFrame
    - min_cols_filled: minimum number of non-null columns to keep row
    - city_to_state: optional dict to map city -> state for row-based imputation
    """
    
    # Drop rows with too few columns filled
    df = df[df.notna().sum(axis=1) >= min_cols_filled]

    # Handle customerid
    if 'customerid' in df.columns:
        df = df[df['customerid'].notna()]  # cannot keep rows without ID

    # Handle Age (Can use ML prediction later)
    if 'age' in df.columns:
        median_age = df['age'].median()
        df['age'] = df['age'].fillna(median_age)

    # Step 4: Handle Gender
    if 'gender' in df.columns:
        missing_ratio = df['gender'].isna().mean()
        if missing_ratio > 0.5:
            df['gender'] = df['gender'].fillna('Unknown')
        else:
            df['gender'] = df['gender'].fillna(df['gender'].mode()[0])

    # Step 5: Handle City, State, Country
    for col in ['city', 'state', 'country']:
        if col in df.columns:
            if col == 'country':
                df[col] = df[col].fillna('Malaysia')
            elif col == 'state':
                # fill based on city mapping if provided
                if city_to_state:
                    df['state'] = df.apply(
                        lambda r: city_to_state.get(r['city'], r['state']) 
                        if pd.isna(r['state']) else r['state'], axis=1
                    )
                # fallback to mode
                df['state'] = df['state'].fillna(df['state'].mode()[0])
            else:  # city
                df[col] = df[col].fillna(df[col].mode()[0])

    # -------------------
    # Optional Step 6: Predictive Imputation (Placeholder)
    # -------------------
    # Here you could add ML-based prediction for age, gender, etc.
    # Example:
    # df['age'] = predictive_age_fill(df)
    # df['gender'] = predictive_gender_fill(df)

    return df

# =============================================(WIP) STAGE 5: OUTLIER DETECTION =============================================

def detect_outliers(df):
    """Detect outliers in DOB and Age"""
    if 'dob' in df.columns:
        df['dob'] = df['dob'].apply(
            lambda x: np.nan if pd.notnull(x) and (date.today().year - x.year > 110 or date.today().year - x.year < 0) else x
        )

    if 'age' in df.columns:
        df['age'] = df['age'].apply(lambda x: np.nan if pd.notnull(x) and (x < 0 or x > 110) else x)
    return df


# ============================================= STAGE 6: DEDUPLICATION =============================================
# This function removes duplicate CustomerIDs and keeps the row with the most non-missing data to preserve the most complete customer record.
def deduplicate_customers(df):
    """Keep the most complete record for each CustomerID"""
    if 'customerid' in df.columns:
        df = (
            df.loc[df.groupby('customerid').apply(lambda x: x.notna().sum(axis=1).idxmax())]
            .reset_index(drop=True)
        )
    return df


# =============== MAIN PIPELINE ===============

def clean_customer_dataset(df):
    df = normalize_columns_name(df)
    df = standardize_customer_id(df)
    df = standardize_dob(df)
    df = derive_age_features(df)
    df = derive_age_group(df)

    df = standardize_gender(df)
    df = standardize_location(df)

    df = handle_missing_values_row_aware(df)
    df = detect_outliers(df)
    df = deduplicate_customers(df)
    return df
